Python Section 2

(Questions in this section are interrelated, so please solve them accordingly.)

Question 9: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-2.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B aframe:


In [1]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(df: pd.DataFrame) -> pd.DataFrame:
    matrix = pd.pivot_table(df, values='distance', index='id_start', columns='id_end', aggfunc='sum')
    matrix = matrix.fillna(0)
    
    # Symmetric matrix for bidirectional routes
    matrix = matrix.add(matrix.T, fill_value=0)
    return matrix


Question 10: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 9. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [2]:
import pandas as pd

def unroll_distance_matrix(df: pd.DataFrame) -> pd.DataFrame:
    unrolled = df.stack().reset_index()
    unrolled.columns = ['id_start', 'id_end', 'distance']
    unrolled = unrolled[unrolled['id_start'] != unrolled['id_end']]
    return unrolled


Question 11: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 10 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [3]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(df: pd.DataFrame, reference_id: int) -> pd.DataFrame:
    ref_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()
    min_threshold = ref_avg_distance * 0.9
    max_threshold = ref_avg_distance * 1.1

    return df.groupby('id_start')['distance'].mean().loc[
        lambda x: (x >= min_threshold) & (x <= max_threshold)
    ].index.tolist()


Question 12: Calculate Toll Rate

Create a function calculate_toll_rate that takes the DataFrame created in Question 10 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [4]:
import pandas as pd

def calculate_toll_rate(df: pd.DataFrame) -> pd.DataFrame:
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}
    
    for vehicle, rate in rate_coefficients.items():
        df[vehicle] = df['distance'] * rate

    return df


Question 13: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 12 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [5]:
import pandas as pd
import datetime

def calculate_time_based_toll_rates(df: pd.DataFrame) -> pd.DataFrame:
    df['start_time'] = pd.to_datetime(df['start_time']).dt.time
    df['end_time'] = pd.to_datetime(df['end_time']).dt.time
    
    def apply_time_discounts(row):
        weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        if row['start_day'] in weekdays:
            if row['start_time'] <= pd.to_datetime('10:00:00').time():
                discount_factor = 0.8
            elif row['start_time'] <= pd.to_datetime('18:00:00').time():
                discount_factor = 1.2
            else:
                discount_factor = 0.8
        else:
            discount_factor = 0.7
        
        return discount_factor

    for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
        df[vehicle] = df.apply(apply_time_discounts, axis=1) * df[vehicle]
    
    return df
